In [18]:
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
import trimesh as tm
import os
from pathlib import Path
import subprocess

import torch
from pointsRefine_emptyUp import meshIt_, vertexKey, makeVoxelDict, voxelGrid2Colors, voxelGrid2Voxels, fillVoxelsFurther2, fillVoxelsFurther, translateOldVoxels2New, fillVoxelMinimalDown, fillVoxelColumns, createVoxelPointsXYDict, appendEmptyPoints, loadPoints, loadEmptyPoints, checkEmptyPoints, appendPoints, appendEmptyPoints, createVoxelPointsXYDict
import voxelstuff

import voxelstuff
import pointCloudOperations
import constructCity_config

debug = constructCity_config.debug

pfad = constructCity_config.pfad
vhacdPath = constructCity_config.vhacdPath
partSize = 6
times = 0

cl = None
pcd = None


breakAll = False



#################################################################################
if True:
    import os
    heights = {}
    minBound__ = np.ones(3) * 9999999
    maxBound__ = np.ones(3) * 0
    for name in os.listdir(pfad+"/blocks"):
        x = int(name.split("x")[0])
        y = int(name.split("x")[1].split("y")[0])
        z = int(name.split("x")[1].split("y")[1].split("z")[0])
        if x < minBound__[0]:
            minBound__[0] = x
        if y < minBound__[1]:
            minBound__[1] = y
        if z < minBound__[2]:
            minBound__[2] = z
        if x > maxBound__[0]:
            maxBound__[0] = x
        if y > maxBound__[1]:
            maxBound__[1] = y
        if z > maxBound__[2]:
            maxBound__[2] = z
        if (x,y) in heights:
            heights[(x,y)] = [min(z,heights[(x,y)][0]),max(z,heights[(x,y)][1])]
        else:
            heights[(x,y)] = [z,z]
    np.save(pfad+"/heights.npy", np.array(list(heights.keys())))
    np.save(pfad+"/heightsValues.npy", np.array(list(heights.values())))
    print(minBound__//partSize)
    print(maxBound__//partSize)
    # -%%
    #for x_mid in range(int(maxBound__[0]-minBound__[0])//partSize):
    #    x_mid = x_mid*partSize+int(minBound__[0])+partSize//2
    #    for y_mid in range(int(maxBound__[1]-minBound__[1])//partSize):
    #        y_mid = y_mid*partSize+int(minBound__[1])+partSize//2
            #for z_mid in range(int(maxBound__[2]-minBound__[2])//partSize):
            #    z_mid = z_mid*partSize+int(minBound__[2])
            # we grab all z to have a bottom
    # -%%
    x_mid = 8
    y_mid = 8
    for ttt2 in range(1):
        #for x_mid in range(int(maxBound__[0]-minBound__[0])//partSize):
        x_mid = x_mid*partSize+int(minBound__[0])+partSize//2
        for ttt in range(1):
            #for y_mid in range(int(maxBound__[1]-minBound__[1])//partSize):
            y_mid = y_mid*partSize+int(minBound__[1])+partSize//2
            if True:
                print("Loading ",x_mid,y_mid)
                alreadyRefined = False
                points = None
                for x in range(-partSize//2+x_mid,partSize//2+x_mid):
                    for y in range(-partSize//2+y_mid,partSize//2+y_mid):
                        for z in range(int(minBound__[2]),int(maxBound__[2]+1)):
                            try:
                                if checkEmptyPoints(x,y,z):
                                    alreadyRefined = True
                                if alreadyRefined:
                                    continue
                                points_ = loadPoints(x,y,z)
                                if points_ is not None:
                                    try:
                                        points = np.concatenate([points,points_],0)
                                    except:
                                        points = points_
                            except: pass # no points to load here
                if points is not None:
                        pcd = o3d.geometry.PointCloud()
                        pcd.points = o3d.utility.Vector3dVector(points[:,:3])
                        pcd.colors = o3d.utility.Vector3dVector(points[:,3:])
                        o3d.visualization.draw_geometries([pcd])
                else:
                    print("no points")

[ 0. -1.  0.]
[27. 34.  5.]
Loading  51 50
